<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/1_2-Easy_NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<h1>Large Language Models Projects</h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>1.2-Create a simple Natural Language to SQL using OpenAI</h2>
    
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
<hr>

Models: gpt-3.5-turbo / gpt-4o-mini

Colab Environment: CPU

Keys:
* NL2SQL
* Code Generation
* Prompt Hardening.

Related article: [How to Create a Natural Language to SQL Translator Using OpenAI API](https://pub.towardsai.net/how-to-create-a-natural-language-to-sql-translator-using-openai-api-e1b1f72ac35a)
____________________
This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (Apress).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

# SQL Generator
A sample of how to build a translator from natural language to SQL:

* GPT 35 / gpt-4o-mini
* OpenAI


In [ ]:
!pip install -q openai==1.1.1
!pip install -q panel

In [ ]:
import openai
import panel as pn
openai.api_key="your-openai-key"
#model = "gpt-3.5-turbo"
model = "gpt-4o-mini"

In [ ]:
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User promnopt.
    context.append({'role':'user', 'content':f"{prompt}."})
    context.append({'role':'system', 'content':f"""Only return SQL Orders.
    If you can't return and SQL order, say sorry, and ask, politely but concisely, for a new question."""})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Undate the panels to shjow the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [ ]:
context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with
this is your SQL, and after that an SQL that can do what the user request.

Your SQL Database is composed by some tables.
Try to Maintain the SQL order simple.
Just after the SQL add a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order
just answer something nice and simple, maximum 10 words, asking him for something that
can be solved with SQL.
"""} ]

context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "string"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "string"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "string"
    }
  ]
}
"""
})

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Order your data…')
button_conversation = pn.widgets.Button(name="generate SQL")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

In [ ]:
#Sample question: "Return the name of the best paid employee"
dashboard